In [6]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2

In [7]:
hoteles = pd.read_csv('hoteles_unicos.csv')
categorias = pd.read_csv('categorias_unicas.csv')

## 1 endpoint

In [14]:
def top_hoteles_por_ubicacion(estado:str, ciudad:str):
    global hoteles,categorias

    estado = estado.upper()
    ciudad = ciudad.title()
    
    hoteles_ubicacion = hoteles[(hoteles['state'] == estado) & (hoteles['city'] == ciudad)]
    if hoteles_ubicacion.empty:
        return {"No se encontraron hoteles en la ubicación proporcionada."}
    else:
        top_hoteles = hoteles_ubicacion.nlargest(5, 'bussines_stars')
        top_hoteles = top_hoteles[['name', 'bussines_stars']]
        return {f'El top de hoteles para tu ubicación es':top_hoteles.to_dict(orient='records')}


In [16]:
top_hoteles_por_ubicacion('CA', 'New OrleaNs')

{'El top de hoteles para tu ubicación es': [{'name': 'Doctor Gumbo Tours',
   'bussines_stars': 5.0},
  {'name': 'St. Philip French Quarter Apts', 'bussines_stars': 4.5},
  {'name': 'House Of Broel', 'bussines_stars': 4.0},
  {'name': 'The Grill Room', 'bussines_stars': 4.0},
  {'name': 'One11 Hotel', 'bussines_stars': 4.0}]}

## 2 endpoint

In [17]:
def distancia_haversine(lat1, lon1, lat2, lon2):
    # Convertir grados a radianes
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Radio de la Tierra en kilómetros
    radio_tierra = 6371.0

    # Diferencia de latitud y longitud
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Calcular la distancia utilizando la fórmula haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distancia = radio_tierra * c

    return distancia

# Función para ordenar el DataFrame categorias en función de la cercanía al local indicado
def ordenar_por_cercania(local):
    #global hoteles, categorias
    global hoteles, categorias
    local = local.title()
    
    # Obtiene las coordenadas del local indicado
    latitud_local, longitud_local, ciudad_local, estado_local = hoteles[hoteles['name'] == local][['latitude', 'longitude','city','state']].values[0]

    # Filtrar las filas que tienen la misma ciudad y estado que el local
    categorias = categorias[(categorias['city'] == ciudad_local) & (categorias['state'] == estado_local)]
    
    
    # Calculamos la distancia entre las coordenadas del local y cada negocio en categorias
    categorias['distancia'] = categorias.apply(lambda row: distancia_haversine(latitud_local, longitud_local, row['latitude'], row['longitude']), axis=1)

    # Ordena categorias por la distancia calculada
    categorias_ordenado = categorias.sort_values(by='distancia')
    categorias_ordenado = categorias_ordenado[['name','categories', 'distancia', 'city', 'state']]
    
    return {f'Los locales mas cercanos son':categorias_ordenado.to_dict(orient='records')}

In [18]:
ordenar_por_cercania('Ghost CiTy Tours Nashville')

{'Los locales mas cercanos son': [{'name': 'Country Music Hall of Fame and Museum',
   'categories': 'Arts & Entertainment, Museums',
   'distancia': 1.1397484665469897,
   'city': 'Nashville',
   'state': 'CA'},
  {'name': 'Marathon Village',
   'categories': 'Party & Event Planning, Event Planning & Services, Home Services, Food, Landmarks & Historical Buildings, Shopping Centers, Real Estate, Convenience Stores, Commercial Real Estate, Shopping, Public Services & Government, Museums, Arts & Entertainment, Venues & Event Spaces, Distilleries',
   'distancia': 1.162564736108249,
   'city': 'Nashville',
   'state': 'CA'},
  {'name': 'Sambuca Restaurant',
   'categories': 'Music Venues, American (New), Arts & Entertainment, Restaurants, Nightlife, Jazz & Blues, Bars',
   'distancia': 1.632240499927073,
   'city': 'Nashville',
   'state': 'CA'},
  {'name': 'Urban Cookhouse',
   'categories': 'American (New), Automotive, Caterers, American (Traditional), Event Planning & Services, Comfort

## 3 endpoint

In [9]:
# Función para ordenar el DataFrame categorias por 'bussines_stars' de mayor a menor, conservando la distancia
def ordenar_por_bussines_stars(local):
    global hoteles, categorias
    local = local.title()
    # Obtenemos las coordenadas del local indicado
    
    latitud_local, longitud_local, ciudad_local, estado_local = hoteles[hoteles['name'] == local][['latitude', 'longitude','city','state']].values[0]
    #latitud_local, longitud_local = hoteles[hoteles['name'] == local][['latitude', 'longitude']].values[0]
    
    # Calculamos la distancia entre las coordenadas del local y cada negocio en categorias
    categorias['distancia'] = categorias.apply(lambda row: distancia_haversine(latitud_local, longitud_local, row['latitude'], row['longitude']), axis=1)

    # Filtrar las filas que tienen la misma ciudad y estado que el local
    #categorias = categorias[(categorias['city'] == ciudad_local) & (categorias['state'] == estado_local)]

    # Ordenamos categorias por 'bussines_stars' de mayor a menor
    categorias_ordenado = categorias.sort_values(by='bussines_stars', ascending=False)

    categorias_ordenado = categorias_ordenado[['name', 'bussines_stars','categories', 'distancia', 'city', 'state']]
    return {f'El top de hoteles para tu ubicación es':categorias_ordenado.to_dict(orient='records')}

In [10]:
ordenar_por_bussines_stars('Ghost City Tours Nashville')

{'El top de hoteles para tu ubicación es': [{'name': 'Midtown Gallery Framers',
   'bussines_stars': 5.0,
   'categories': 'Art Galleries, Framing, Arts & Entertainment, Arts & Crafts, Shopping',
   'distancia': 5.929689258818121,
   'city': 'Nashville',
   'state': 'CA'},
  {'name': 'Richland Creek Greenway',
   'bussines_stars': 4.5,
   'categories': 'Hiking, Active Life, Parks',
   'distancia': 6.069372627946706,
   'city': 'Nashville',
   'state': 'CA'},
  {'name': 'Urban Cookhouse',
   'bussines_stars': 4.0,
   'categories': 'American (New), Automotive, Caterers, American (Traditional), Event Planning & Services, Comfort Food, Beer, Wine & Spirits, Parking, Sandwiches, Beer Bar, Vegetarian, Breakfast & Brunch, Wine Bars, Restaurants, Bars, Food, Nightlife',
   'distancia': 2.1364876779839155,
   'city': 'Nashville',
   'state': 'CA'},
  {'name': 'The Local',
   'bussines_stars': 4.0,
   'categories': 'Arts & Entertainment, Nightlife, Music Venues, Jazz & Blues, Bars, Restaurants, 

## Test

In [3]:
hoteles.head(1)

,business_id,name,categories,city,latitude,longitude,bussines_stars,state
0,hmO76DoSlxEUPJ2NN3K9aw,Fly Away Airport Parking,"Airport Shuttles, Hotels & Travel, Transportat...",Nashville,36.103278,-86.667719,2.5,CA


In [4]:
hoteles[hoteles['city'] == 'New Orleans']

,business_id,name,categories,city,latitude,longitude,bussines_stars,state
11,iJ3A49gtUPspc7nd4Ov2HA,Escape My Room,"Local Flavor, Challenge Courses, Event Plannin...",New Orleans,29.947357,-90.067930,5.0,AZ
13,cxB31SdS6anItwCmTOKtNg,International House Hotel,"Hotels & Travel, Hotels, Event Planning & Serv...",New Orleans,29.951224,-90.068947,3.5,AZ
15,2XEf2xOj-iZImMeWELkQFw,Place d'Armes Hotel,"Event Planning & Services, Hotels, Hotels & Tr...",New Orleans,29.958753,-90.063406,4.0,NV
21,kYgMSD2obyOgORgd3RSE_w,Mr John's Steakhouse,"Seafood, Fitness & Instruction, Hotels, Hotels...",New Orleans,29.934921,-90.080256,4.0,NV
23,hAZlcREMAHe0z9GbMX-Faw,Hyatt Centric French Quarter New Orleans,"Hotels & Travel, Event Planning & Services, Ho...",New Orleans,29.955037,-90.070057,4.0,NV
31,EEbKsmWBhkjWEzEFWsnEZA,Le Méridien New Orleans,"Event Planning & Services, Hotels & Travel, Ho...",New Orleans,29.948997,-90.066320,3.0,AZ
44,5C3Xq33S5VgLun8vxTxMqA,Two Chicks Walking Tours,"Historical Tours, Arts & Entertainment, Archit...",New Orleans,29.930713,-90.076524,5.0,AZ
51,KsKXCCfAqSav1PnlyEreaA,Hilton Garden Inn New Orleans Convention Center,"Event Planning & Services, Hotels & Travel, Ho...",New Orleans,29.941276,-90.066031,3.0,NV
59,fla7uhSa82DyFTMxiO53ng,Four Points by Sheraton French Quarter,"Hotels & Travel, Event Planning & Services, Ve...",New Orleans,29.957798,-90.066612,3.5,CA
73,yi9udL5fcmBjwcBcygv51Q,Canal Place,"Shoe Stores, Fashion, Caterers, Shopping, Hote...",New Orleans,29.950999,-90.064996,3.5,CA
